In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.cm as cm 
import seaborn as sns
import pandas as pd
from scipy import signal

import winsound as sd
import pickle
import os
import sys

In [ ]:
# Load
cd = os.getcwd()
path = os.path.dirname(cd)

with open(path+'//Data_DG.pkl', 'rb') as file:
    # Patterns in input layer
    Data = pickle.load(file)   
    # The number of data
    Datalen = pickle.load(file)    
    # Patterns of granule cells
    Data_DG = pickle.load(file)
    
    # Lengths of each patterns of granule cells
    Data_DG_len = pickle.load(file)
    Data_idx = pickle.load(file) # indices of patterns having proper lengths
    DataX_idx = pickle.load(file) # indices of patterns having proper lengths
    
    # input/output overlap values and indices
    OverI = pickle.load(file)
    OverO = pickle.load(file)
    Pair_idx = pickle.load(file)
    
    # input/output overlap values and indices for inputs having proper lengths
    OverI_v2 = pickle.load(file)
    OverO_v2 = pickle.load(file)
    Pair_idx_v2 = pickle.load(file)

In [ ]:
# 중복 데이터 제거
Data_set = []
for n in range(Datalen):
    Data_set.append(set(np.where(Data[:,n] !=0)[0]))
Repeated_data = []
for i, comp in enumerate(Data_set):
    for j, comp2 in enumerate(Data_set):
        if i < j:
            if comp == comp2:
                Repeated_data.append(j)
                
Data_final_idx = set(range(Datalen))-set(Repeated_data)
Datalen_final = len(Data_final_idx)
Data_final = np.zeros((16,Datalen_final))
for i, comp in enumerate(Data_final_idx):
    Data_final[:,i] = Data[:,comp]
print(Datalen_final)

In [ ]:
with open(path+'//Data_final.pkl', 'rb') as file:
    # Filtered by proper length
    Filt_idata = pickle.load(file) # patterns in input layer
    Filt_gdata = pickle.load(file) # patterns of granule cells
    Filt_glen = pickle.load(file) # lengths of each patterns of granule cells
    F_DataLen = pickle.load(file) # the number of data
    
    # Index of CA3 patterns
    Af_py = pickle.load(file) # Pyramidal neurons
    Af_in = pickle.load(file) # Inhibitory neurons
    
    # 
    In_py_list = pickle.load(file)
    In_in_list = pickle.load(file)

In [ ]:
def cal_overlap(total,A,B):
    if (len(A)+len(B)) !=0:
        output = len(A-(total- B))*2 / (len(A)+len(B))
    else:
        output = 0
    return output

In [ ]:
Overlap_ascending_I = []
Overlap_ascending_O = []
Classfy_num = 25
for n in range(Classfy_num+1):
    Overlap_ascending_I.append([])
    Overlap_ascending_O.append([])
    
for i, comp in enumerate(OverI):
    if comp == 0:
        Overlap_ascending_I[0].append(comp)
        Overlap_ascending_O[0].append(OverO[i])
    for n in range(Classfy_num):    
        if (1/Classfy_num)*n < comp <= (1/Classfy_num)*n+(1/Classfy_num):
            Overlap_ascending_I[n+1].append(comp)
            Overlap_ascending_O[n+1].append(OverO[i])

In [ ]:
OverI_block = []
OverO_block = []
for i, comp in enumerate(Overlap_ascending_I):
    OverI_block.append(np.mean(comp))
    OverO_block.append(np.mean(Overlap_ascending_O[i]))
    
DeltaI_block = []
DeltaO_block = []
DeltaO_block_std = []
for i, comp in enumerate(Overlap_ascending_I):
    DeltaI_block.append(1-np.mean(comp))
    DeltaO_block.append(1-np.mean(Overlap_ascending_O[i]))
    DeltaO_block_std.append(np.std(np.array(Overlap_ascending_O[-1])))

x = np.linspace(0,1,100)
fig = plt.subplot(111)
plt.scatter(DeltaI_block, DeltaO_block, color='#423121', label='DG')
#plt.errorbar(DeltaI_block, DeltaO_block, yerr=DeltaO_block_std, fmt="o")
plt.plot(x,x, color='black')
#plt.xlim([0,1])
#plt.ylim([0,1.2])
plt.xlabel('$\Delta$ Input')
plt.ylabel('$\Delta$ Output (DG)')
plt.axis("square")
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
plt.savefig('Separation performance of DG.pdf', bbox_inches='tight')
plt.show()

In [ ]:
data_dic = {}
data_dic['$\Delta$ Input'] = []
data_dic['$\Delta$ Output (DG)'] = []
for i, comp in enumerate(DeltaI_block):
    for m in range(len(Overlap_ascending_O[i])):
        data_dic['$\Delta$ Input'].append(comp)
    data_dic['$\Delta$ Output (DG)'] += list(1-np.array(Overlap_ascending_O[i]))

df = pd.DataFrame(data_dic)    
    
fig = plt.subplot(111)
sns.set(rc={'figure.figsize':(5, 5)})
sns.set_theme(style='white')
ax = sns.lineplot(data = df, x='$\Delta$ Input', y='$\Delta$ Output (DG)', color='#423121', marker='o', markersize=7.5, linewidth=2.5)
ax.tick_params(axis='both', direction='out', length=3.5, width=0.75, colors='black')
x = np.linspace(0,1,100)
plt.plot(x,x, color='black')
plt.xlim([0,1.05])
plt.ylim([0,1.05])
plt.xlabel('$\Delta$ Input', fontsize=16)
plt.ylabel('$\Delta$ DG output', fontsize=16)
plt.title('Pattern separation performance', fontsize=16)
fig.spines['top'].set_visible(False)
fig.spines['right'].set_visible(False)
fig.set_aspect(1.0 / fig.get_data_ratio(), adjustable="box")
plt.savefig('Separation performance of DG.pdf', bbox_inches='tight')
plt.show()

In [ ]:
OverI_new = np.array([])
OverO_new = np.array([])

for i, comp in enumerate(OverI):
    if comp != 0:
        OverI_new = np.append(OverI_new, comp)
        OverO_new = np.append(OverO_new, OverO[i])

In [ ]:
palette = sns.color_palette('cool')
palette

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(5,5))
sns.set_theme(style='white')
fig.subplots_adjust(hspace = 0.1)
ax1 = sns.histplot(OverI_v2, binwidth= 0.05, kde=False, stat='percent', color=palette[1], multiple='dodge', ax=ax1)
ax1 = sns.histplot(OverO_v2, binwidth= 0.05, kde=False, stat='percent', color=palette[-3], multiple='dodge', ax=ax1)
ax2 = sns.histplot(OverI_v2, binwidth= 0.05, kde=False, stat='percent', color=palette[1], multiple='dodge', ax=ax2)
ax2 = sns.histplot(OverO_v2, binwidth= 0.05, kde=False, stat='percent', color=palette[-3], multiple='dodge', ax=ax2)

ax2.set_xlabel('Similarity between engrams', fontsize=16)
ax1.set_ylabel("")
ax2.set_ylabel("")

ax1.set_ylim(60, 100)
ax2.set_ylim(0, 20)

fig.text(-0.02, 0.5, "Information pair distribution (%)", va='center', rotation = 'vertical', fontsize=16)
fig.text(0.2, 0.95, 'Distribution of engram pairs', va='center', fontsize=16)
ax1.get_xaxis().set_visible(False)

d = .7    # how big to make the diagonal lines in axes coordinates

kwargs = dict(marker=[(-1, -d), (1, d)], markersize=15, linestyle="none", color='k', clip_on=False)

ax1.plot([0.05, 0.09], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0.05, 0.09], [1, 1], transform=ax2.transAxes, **kwargs)
ax1.legend(handles=[Rectangle((0,0), 0, 0, color = palette[1], label="Input (sEC)"), Rectangle((0,0), 0, 0, color = palette[-3], label="Output (DG)")],  loc='upper right', frameon=False)

ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)

ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax2.spines['right'].set_visible(False)
#plt.savefig('Histogram for DG overlap.pdf', bbox_inches='tight')
plt.show()